In [37]:
import pandas as pd

import numpy as np


In [38]:
expid = 134
caseStudyIdx = 16
predDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{caseStudyIdx}/case_study_pred.csv"
)
gtDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{caseStudyIdx}/case_study_gt.csv"
)

visionPredDf1 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv")
visionPredDf2 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
visionPredDf3 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")
visionPredDf = pd.concat([visionPredDf1, visionPredDf2,visionPredDf3 ])
majorParts = [x.replace("vision_", "") for x in  visionPredDf.columns]
majorParts

['Unnamed: 0',
 'CaseID',
 'bonnet',
 'bumper_front',
 'engine',
 'fender_front_lh',
 'front_panel',
 'grille',
 'headlamp_lh',
 'windscreen_front',
 'bumper_rear',
 'rear_compartment',
 'rear_panel',
 'rear_quarter_rh',
 'tail_lamp_rh',
 'windscreen_rear',
 'rear_quarter_lh',
 'tail_lamp_lh',
 'fender_front_rh',
 'headlamp_rh']

In [39]:
len(predDf)

29

In [40]:
targetCols = [x for x in predDf.columns if "0" not in x and x != "CaseID"]
allRows = []
for c in targetCols:
    for caseId in predDf["CaseID"].unique().tolist():
        visionPred = -1
        if(c in majorParts):
            visionPred = visionPredDf[visionPredDf["CaseID"] == caseId][f"vision_{c}"].item()
            

        pred = predDf[predDf["CaseID"] == caseId][c].item()
        gt = gtDf[gtDf["CaseID"] == caseId][c].item()
        fp = 1 if gt == 0 and pred == 1 else 0
        fn = 1 if gt == 1 and pred == 0 else 0
        allRows.append(
            {
                "CaseID" : caseId,
                "part" : c,
                "v_pred":visionPred,
                "pred" : pred,
                "gt" : gt,
                "fp" : fp, 
                "fn" : fn,
                "inspect" : None,
                "remarks" : None

            }
        )
caseStudyInspectDf = pd.json_normalize(allRows)
caseStudyInspectDf.sort_values(by = ["CaseID", "part"], inplace=True)

In [41]:
caseStudyInspectDf

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
0,10071959,bonnet,1,1,1,0,0,None,None
29,10071959,bumper_front,1,1,1,0,0,None,None
145,10071959,bumper_rear,0,0,0,0,0,None,None
174,10071959,engine,1,1,1,0,0,None,None
232,10071959,fender_front_lh,1,1,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...
492,13194275,rear_quarter_rh,0,0,0,0,0,None,None
347,13194275,tail_lamp_lh,0,0,0,0,0,None,None
376,13194275,tail_lamp_rh,0,0,0,0,0,None,None
405,13194275,windscreen_front,0,0,0,0,0,None,None


In [42]:
len(caseStudyInspectDf["part"].unique())

18

In [43]:
majorParts = 25 * 30
majorParts

750

In [44]:
154 / majorParts

0.20533333333333334

In [45]:
caseStudyWrong = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["gt"] )
                                    & (caseStudyInspectDf["v_pred"] != -1)
                                    
                                    ]
caseStudyWrong

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
232,10071959,fender_front_lh,1,1,0,1,0,None,None
261,10071959,fender_front_rh,1,1,0,1,0,None,None
116,10071959,headlamp_rh,1,1,0,1,0,None,None
464,10071959,rear_quarter_rh,1,1,0,1,0,None,None
1,10245671,bonnet,0,0,1,0,1,None,None
...,...,...,...,...,...,...,...,...,...
491,13081007,rear_quarter_rh,1,1,0,1,0,None,None
404,13081007,windscreen_front,1,1,0,1,0,None,None
260,13194275,fender_front_lh,1,1,0,1,0,None,None
289,13194275,fender_front_rh,1,1,0,1,0,None,None


In [46]:
len(caseStudyWrong) / len(caseStudyInspectDf)

0.15900383141762453

In [47]:

caseStudyWrongVis = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["v_pred"]) & (caseStudyInspectDf["v_pred"] != -1) ]
caseStudyWrongVis

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks


In [48]:
# majorPartDf = caseStudyInspectDf[caseStudyInspectDf["v_pred"] != - 1]
majorPartDf = caseStudyInspectDf

In [49]:
majorPartDf["part"].unique()

array(['bonnet', 'bumper_front', 'bumper_rear', 'engine',
       'fender_front_lh', 'fender_front_rh', 'front_panel', 'grille',
       'headlamp_lh', 'headlamp_rh', 'rear_compartment', 'rear_panel',
       'rear_quarter_lh', 'rear_quarter_rh', 'tail_lamp_lh',
       'tail_lamp_rh', 'windscreen_front', 'windscreen_rear'],
      dtype=object)

In [50]:
fpr = len(majorPartDf[(majorPartDf["pred"]  == 1) & (majorPartDf["gt"]  == 0) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 0)])
fpr

0.12151898734177215

In [51]:
fnr = len(majorPartDf[(majorPartDf["pred"]  == 0) & (majorPartDf["gt"]  == 1) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 1)])
fnr

0.2755905511811024

In [52]:
acc = len(majorPartDf[(majorPartDf["pred"]  == majorPartDf["gt"])]) / len(majorPartDf)
acc

0.8409961685823755

In [53]:
allSubsetAcc = []
for caseId in majorPartDf["CaseID"].unique().tolist():
    rows = majorPartDf[majorPartDf["CaseID"] == caseId]
    correct = rows[rows["pred"] == rows["gt"]]
    subset_acc = len(correct) / len(rows)
    allSubsetAcc.append(subset_acc)
    
    # print(subset_acc)
print(np.mean(allSubsetAcc))

0.8409961685823754


In [54]:
caseStudyInspectDf.to_csv("./case_study_result.csv")